<a href="https://colab.research.google.com/github/VellummyilumVinoth/Aspect_based_sentimental_analysis/blob/main/content_based_filtering_for_a_new_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [90]:
# Step 1: Load the dataset

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/zidan/aspect_extracted_file_info.csv')


In [91]:
df.head(10)

,product_name,review,preprocess_text,preprocess_text1,aspects,price,quality,delivery cost,packaging,gender,age_category,area
0,"Salate Cordless Electric Spin Scrubber, Electr...",I absolutely love this wireless electric spin ...,i absolutely love this wireless electric spin ...,"['absolutely', 'love', 'wireless', 'electric',...",['quality'],0,1,0,0,Male,21,Southern
1,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,i used this to clean my bathroom and this work...,"['used', 'clean', 'bathroom', 'works', 'wonder...",['quality'],0,1,0,0,Female,55,Uva
2,"Salate Cordless Electric Spin Scrubber, Electr...",This electric brush makes cleaning so much eas...,this electric brush makes cleaning so much eas...,"['electric', 'brush', 'makes', 'cleaning', 'mu...",['delivery cost'],0,0,1,0,Female,68,Eastern
3,"Salate Cordless Electric Spin Scrubber, Electr...",This Electric spin scrubber is perfect! Use it...,this electric spin scrubber is perfect use it ...,"['electric', 'spin', 'scrubber', 'perfect', 'u...",['quality'],0,1,0,0,Other,54,Eastern
4,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,i dont like cleaning but this machine help me ...,"['dont', 'like', 'cleaning', 'machine', 'help'...",['quality'],0,1,0,0,Male,32,Northern
5,"Salate Cordless Electric Spin Scrubber, Electr...",It has really made my life so much easy! Its v...,it has really made my life so much easy its ve...,"['really', 'made', 'life', 'much', 'easy', 'ea...",['quality'],0,1,0,0,Other,73,Northern
6,"Salate Cordless Electric Spin Scrubber, Electr...",So far I've used this powered scrubber a coupl...,so far ive used this powered scrubber a couple...,"['far', 'ive', 'used', 'powered', 'scrubber', ...","['quality', 'delivery cost']",0,1,1,0,Other,64,Eastern
7,"Salate Cordless Electric Spin Scrubber, Electr...",I did not like to clean the kitchen. This mach...,i did not like to clean the kitchen this machi...,"['like', 'clean', 'kitchen', 'machine', 'helpe...",['quality'],0,1,0,0,Female,60,Northern
8,"Charmin Ultra Strong Clean Touch Toilet Paper,...","Ladies and gentlemen, get ready for a review t...",ladies and gentlemen get ready for a review th...,"['ladies', 'gentlemen', 'get', 'ready', 'revie...","['quality', 'delivery cost', 'packaging']",0,1,1,1,Female,36,Northern
9,"Charmin Ultra Strong Clean Touch Toilet Paper,...","Oh, Ultra Strong Toilet Paper, how you do\nPro...",oh ultra strong toilet paper how you do\nprote...,"['oh', 'ultra', 'strong', 'toilet', 'paper', '...",['quality'],0,1,0,0,Female,58,Northern


In [93]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Extract the features
reviews = df["review"]
product_names = df["product_name"]
age = df["age_category"]
gender = df["gender"]
area = df["area"]

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform the reviews into TF-IDF vectors
review_vectors = vectorizer.fit_transform(reviews)

# Calculate the cosine similarity between all pairs of products
product_similarity = cosine_similarity(review_vectors)

# Create a function to recommend products to a user
def recommend_products(user_age, user_gender, user_area):
    # Validate age input
    if user_age < 18 or user_age > 75:
        raise ValueError("Age should be between 18 and 75.")
        
    # Validate gender input
    valid_genders = ["Male", "Female"]
    if user_gender not in valid_genders:
        raise ValueError("Invalid gender. Please enter 'Male' or 'Female' or 'Other'.")
        
    # Validate area input
    valid_areas = ["Central", "Eastern", "Northern", "North Central", "North Western", "Sabaragamuwa", "Southern", "Uva", "Western"]
    if user_area not in valid_areas:
        raise ValueError("Invalid area. Please enter a valid province name.")

    # Filter products that match the user's age, gender, and area
    filtered_indices = np.where((age == user_age) & (gender == user_gender) & (area == user_area))[0]

    # Calculate the similarity scores for the filtered products
    similarity_scores = np.mean(product_similarity[filtered_indices], axis=0)

    # Sort the products by their similarity to the user
    similar_products = np.argsort(similarity_scores)[::-1]

    # Initialize set to keep track of recommended product names
    recommended_set = set()

    # Return the top 10 unique products with recommendation rank numbers
    recommendations = []
    for rank, product in enumerate(similar_products):
        if len(recommendations) >= 10:
            break
        product_name = product_names[product]
        if product_name not in recommended_set:
            recommendations.append((rank + 1, product_name))
            recommended_set.add(product_name)
    return recommendations

# Recommend products to a user
user_age = 34
user_gender = "Female"
user_area = "Western"

try:
    recommended_products = recommend_products(user_age, user_gender, user_area)
    # Print the recommended products with rank numbers
    for rank, product in recommended_products:
        print(f"Rank {rank}: {product}")
except ValueError as e:
    print("Error:", str(e))


Rank 1: Amazon Basics Non-Scratch Sponges, 6-Pack
Rank 2: Topo Bear Disposable Toilet Brush Holder Set with 40 Toilet Brush Refills Disposable Cleaning System Kit for Bathroom Toilet Bowl Cleaner
Rank 3: Flowtron BK-80D 80-Watt Electronic Insect Killer, 1-1/2 Acre Coverage , Black
Rank 4: LiBa Electric Bug Zapper, Indoor Insect Killer - (2) Extra Replacement Bulbs - Fly, Mosquito Killer and Repellent - Lightweight, Powerful 2800V Grid, Easy-to-Clean, with a Removable Washable Tray.
Rank 5: Kleenex Expressions Disposable Paper Hand Towels, Paper Hand Towels for Bathroom, 6 Boxes, 60 Hand Towels per Box (360 Total Tissues)
Rank 6: Swedish Wholesale Swedish Dish Cloths for Kitchen- 10 Pack Reusable Paper Towels for Counters & Dishes - Eco Friendly Cellulose Sponge Cloth - Assorted
Rank 7: Panasonic BK-3MCCA8BA eneloop AA 2100 Cycle Ni-MH Pre-Charged Rechargeable Batteries, 8-Battery Pack
Rank 8: Downy Wrinkle Release Combo Pack Crisp Linen Scent - (2) 16.9 Ounce + (1) 3 Ounce
Rank 9: OXO 